In [17]:
#Params 
#KNMI data
param_stationID = '260'
param_from_date = '2006'
param_to_date = '2025' 
param_vars = "average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation,
                average_winddirection, relative_humidity, minimum_humidity, maximum_humidity"

param_stationID_spinup = '260'
param_from_date_spinup = '1995'
param_to_date_spinup = '2005' 
param_vars_spinup = "average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation,
                    average_winddirection, relative_humidity, minimum_humidity, maximum_humidity"

#Climate generator 
param_climate_time_series = 'Daily_SequencedYears'
param_spinup_climate_time_series = 'Daily_SequencedYears'

#Biomass initialization file 
param_timestep = '5'

#Scenario file 
param_duration = '100'
param_cell_length = '100'

In [18]:
#KNMI data retriever
# ---
# NaaVRE:
#  cell:
#   params:
#    - param_stationID:
#       type: String
#       default_value: '260'
#    - param_from_date:
#       type: String
#       default_value: '2006'
#    - param_to_date:
#       type: String
#       default_value: '2025'
#    - param_vars: 
#       type: String
#       default_value: 'average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation, average_winddirection, relative_humidity, minimum_humidity, maximum_humidity'
#    - param_stationID_spinup:
#       type: String
#       default_value: '260'
#    - param_from_date_spinup:
#       type: String
#       default_value: '1995'
#    - param_to_date_spinup:
#       type: String
#       default_value: '2005'
#    - param_vars_spinup: 
#       type: String
#       default_value: 'average_windspeed, average_temperature, minimum_temperature, maximum_temperature, total_precipitation, average_winddirection, relative_humidity, minimum_humidity, maximum_humidity'
#   outputs:
#    - KNMI_daily_datafile: String
#    - KNMI_spinup_datafile: String
# ...

#------------------
##Install packages
install.packages("readr")
install.packages("dplyr")
install.packages("tidyr")
install.packages("stringr")
install.packages("lubridate")

#Load packages
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(lubridate)

#Create directory
dir.create("/tmp/data")
#-------------------

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID == "" ||
    param_from_date == "" ||
    param_to_date == "" ||
    length(param_vars) == 0) {
  stop("All parameters must be provided by the user.")
}

#Split climate variables to make columns and rename so that function can use set parameter values 
param_vars <- unlist(strsplit(param_vars, ",\\s*"))
stationID  <- param_stationID
from       <- param_from_date
to         <- param_to_date

param_vars[param_vars == "average_windspeed"]     <- "FG"
param_vars[param_vars == "average_temperature"]   <- "TG"
param_vars[param_vars == "minimum_temperature"]   <- "TN"
param_vars[param_vars == "maximum_temperature"]   <- "TX"
param_vars[param_vars == "total_precipitation"]   <- "RH"
param_vars[param_vars == "average_winddirection"] <- "DDVEC"
param_vars[param_vars == "relative_humidity"]      <- "UG"
param_vars[param_vars == "minimum_humidity"]      <- "UN"
param_vars[param_vars == "maximum_humidity"]      <- "UX"

#Function to get the daily KNMI data, includes checks whether the dates are filled in correctly and logically 
get_data <- function(stationID, from, to, vars) {
  
  #Date parsing - checking whether logical 
  if (!is.character(from) ||
      !is.character(to) ||
      stringr::str_length(from) %% 2 == 1 ||
      stringr::str_length(to) %% 2 == 1) {
    stop(
      "The values for 'from' and 'to' must be strings in the format 'YYYY', 'YYYYMM' or 'YYYYMMDD'."
    )
  }
  
  if (stringr::str_length(from) == 6) {
    from_date <- paste0(from, "01")
  } else if (stringr::str_length(from) == 4) {
    from_date <- paste0(from, "0101")
  } else {
    from_date <- from
  }
  
  if (stringr::str_length(to) == 8) {
    to_date <- to
  } else {
    if (stringr::str_length(to) == 6) {
      to <- paste0(to, "01")
    } else if (stringr::str_length(to) == 4) {
      to <- paste0(to, "1231")
    }
    
    to_date <- lubridate::ymd(to) %>%
      lubridate::ceiling_date(unit = "month") - 1
    
    to_date <- gsub("-", "", as.character(to_date))
  }
  
  if (as.numeric(to_date) < as.numeric(from_date)) {
    stop("'from' must be earlier than or equal to 'to'")
  }
  
  # Station validation - check whether station can be selected. The list contains all the KNMI weather stations that collect daily data 
  selectable_stations <- c("260", "275", "209", "210", "215", "225", "229", "235", "240", "242", "248", "249", "251", "257", "258", "260", "265", 
                           "267", "269", "270", "273", "275", "277", "278", "279", "280", "283", "285", "286", "290", "308", "310", "311", "312", 
                           "313", "315", "316", "319", "323", "324", "330", "331", "340", "343", "344", "348", "350", "356", "370", "375", "377",
                           "380", "391", "392")  
  stationID <- as.character(stationID)
  
  if (!stationID %in% selectable_stations) {
    stop(
      paste0(
        "Invalid stationID. Choose one of: ",
        paste(selectable_stations, collapse = ", ")
      )
    )
  }
  
  #Build URL to retrieve the KNMI data from the website 
  baseURL <- "https://www.daggegevens.knmi.nl/klimatologie/daggegevens"
  URL <- paste0(
    baseURL,
    "?start=", from_date,
    "&end=", to_date,
    "&stns=", stationID,
    "&ALL"
  )
  
  #Download data from KNMI 
  data_daily <- readr::read_csv(
    URL,
    col_names = FALSE,
    comment = "#",
    show_col_types = FALSE
  ) %>%
    dplyr::as_tibble()
  
  colnames(data_daily) <- c(
    "STN", "YYYYMMDD", "FG", "TG", "TN",
    "TX", "RH", "DDVEC", "UG", "UX", "UN"
  )
  
  # Select only requested variables as indicated when parameters were set 
  data_daily <- data_daily %>%
    tidyr::drop_na(dplyr::all_of(vars)) %>%
    select(dplyr::all_of(c("YYYYMMDD", vars)))
  
  # Scaling because KNMI works in 0.1
  if ("FG" %in% vars) data_daily$FG <- data_daily$FG / 10
  if ("TG" %in% vars) data_daily$TG <- data_daily$TG / 10
  if ("TN" %in% vars) data_daily$TN <- data_daily$TN / 10
  if ("TX" %in% vars) data_daily$TX <- data_daily$TX / 10
  if ("RH" %in% vars) data_daily$RH <- data_daily$RH / 10
  
  
  return(data_daily)
}

##Part2: For KNMI spinup data ##

#Parameter validation - check whether all parameters were filled in by user
if (param_stationID_spinup == "" ||
    param_from_date_spinup == "" ||
    param_to_date_spinup == "" ||
    length(param_vars_spinup) == 0) {
  stop("All parameters must be provided by the user.")
}

#Split climate variables to make columns and rename so that function can use set parameter values 
param_vars_spinup <- unlist(strsplit(param_vars_spinup, ",\\s*"))
stationID  <- param_stationID_spinup
from       <- param_from_date_spinup
to         <- param_to_date_spinup

param_vars_spinup[param_vars_spinup == "average_windspeed"]     <- "FG"
param_vars_spinup[param_vars_spinup == "average_temperature"]   <- "TG"
param_vars_spinup[param_vars_spinup == "minimum_temperature"]   <- "TN"
param_vars_spinup[param_vars_spinup == "maximum_temperature"]   <- "TX"
param_vars_spinup[param_vars_spinup == "total_precipitation"]   <- "RH"
param_vars_spinup[param_vars_spinup == "average_winddirection"] <- "DDVEC"
param_vars_spinup[param_vars_spinup == "relative_humidity"]      <- "UG"
param_vars_spinup[param_vars_spinup == "minimum_humidity"]      <- "UN"
param_vars_spinup[param_vars_spinup == "maximum_humidity"]      <- "UX"

#### Part 3: Run the functions twice for daily and spinup 
KNMI_daily <- get_data(
    stationID = param_stationID,
    from      = param_from_date,
    to        = param_to_date,
    vars      = param_vars
)

KNMI_spinup <- get_data(
  stationID = param_stationID_spinup,
  from      = param_from_date_spinup,
  to        = param_to_date_spinup,
  vars      = param_vars_spinup
)


KNMI_daily_datafile <- "/tmp/data/KNMI_daily.csv"
write.csv(KNMI_daily, 
          file = KNMI_daily_datafile, 
          row.names = FALSE)

KNMI_spinup_datafile <- "/tmp/data/KNMI_spinup.csv"
write.csv(KNMI_spinup, 
          file = KNMI_spinup_datafile, 
          row.names = FALSE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘lubridate’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Warning message in dir.create("/tmp/data"):
“'/tmp/data' already exists”


In [15]:
# KNMI data preprocessor
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - KNMI_daily_datafile: String
#    - KNMI_spinup_datafile: String
#   outputs:
#    - Preprocessed_KNMI_datafile: String
#    - Preprocessed_KNMI_spinup_datafile: String
#   dependencies:
#    - name: dplyr
#    - name: stringr
#    - name: lubridate
# ...

#Read the file 
KNMI_daily <- readr::read_csv(KNMI_daily_datafile)
KNMI_spinup <- readr::read_csv(KNMI_spinup_datafile)

#Reordering the date from YYYYMMDD to Year, Month, and Day in separate columns
Preprocessed_KNMI_daily <- KNMI_daily %>%
  dplyr::mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  dplyr::mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  dplyr::select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  dplyr::rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
tidyr::pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "eco1"
  ) %>%
  dplyr::select(Year, Month, Day, Variable, eco1)

## KNMI spinup data ##
#Reordering the date from YYYYMMDD to Year, Month, and Day 
Preprocessed_KNMI_spinup <- KNMI_spinup %>%
 dplyr::mutate(YYYYMMDD = as.character(YYYYMMDD)) %>%
  dplyr::mutate(
    date  = ymd(YYYYMMDD),
    Year  = year(date),
    Month = month(date),
    Day   = day(date)
  ) %>%
  dplyr::select(-YYYYMMDD, -date) %>%
#Renaming KNMI variables to match LANDIS-II names   
  dplyr::rename(any_of(c(
    windSpeed = "FG",
    temp      = "TG",
    mintemp   = "TN",
    maxtemp   = "TX",
    precip    = "RH",
    RH        = "UG",
    maxRH     = "UX",
    minRH     = "UN"
  ))) %>%
 #Reorganize and rename columns 
  tidyr::pivot_longer(
    cols = -c(Year, Month, Day),
    names_to = "Variable",
    values_to = "eco1"
  ) %>%
  dplyr::select(Year, Month, Day, Variable, eco1)

Preprocessed_KNMI_daily_datafile  <- "/tmp/data/Preprocessed_KNMI_daily.csv"
Preprocessed_KNMI_spinup_datafile <- "/tmp/data/Preprocessed_KNMI_spinup.csv"

write.csv(Preprocessed_KNMI_daily, file = Preprocessed_KNMI_daily_datafile, row.names = FALSE)
write.csv(Preprocessed_KNMI_spinup, file = Preprocessed_KNMI_spinup_datafile, row.names = FALSE)

Rows: 7305 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): YYYYMMDD, FG, TG, TN, TX, RH, DDVEC, UG, UN, UX

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 4017 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): YYYYMMDD, FG, TG, TN, TX, RH, DDVEC, UG, UN, UX

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# Species trait data retriever
# Install and load packages
install.packages("dplyr")
install.packages("stringr")
install.packages("tidyr")
install.packages("readr")
library(dplyr)
library(stringr)
library(tidyr)
library(readr)

# Location data sources
input_path <- file.path("/home/jovyan/Cloud Storage/naa-vre-public-data/vl-veluwe-forest-model/")

#retrieve species trait data
#load and prepare TRY database for biomass succession
TRY_biomass <- read.delim(paste0(input_path, "try-data.txt"), fileEncoding = "latin1", dec = ".", quote = "", stringsAsFactors = FALSE)
TRY_biomass <- TRY_biomass %>%
  dplyr::select(-"X") %>%
  dplyr::mutate(
    SpeciesCode = stringr::str_split(.data$AccSpeciesName, " ", simplify = TRUE) %>%
      apply(1, function(x) paste0(stringr::str_sub(x[1], 1, 4), stringr::str_sub(x[2], 1, 4))),
    
    SpeciesCode = dplyr::case_when(
      .data$SpeciesCode %in% c("Querrobu", "Querpetr") ~ "Querspec",
      .data$SpeciesCode %in% c("Betupend", "Betupube") ~ "Betuspec",
      TRUE ~ .data$SpeciesCode
    )
  )

In [5]:
# Species trait data preprocessor
#function to easily min, mean or max a value for a tree trait in TRY    
summarise_by_species <- function(data, stat = c("min", "max", "mean"), name = "result") {
  stat <- match.arg(stat)  # Validate input
  name_sym <- dplyr::sym(name)    # Convert name to symbol for tidy evaluation
  
  data %>%
    dplyr::group_by(.data$SpeciesCode) %>%
    dplyr::summarise(
      !!name_sym := dplyr::case_when(
        stat == "min"  ~ min(.data$OrigValueStr, na.rm = TRUE),
        stat == "max"  ~ max(.data$OrigValueStr, na.rm = TRUE),
        stat == "mean" ~ mean(.data$OrigValueStr, na.rm = TRUE)
      ),
      .groups = "drop"
    )
}

# ---- core species data ----
#is needed for every succession extension
    
##prepare TRY database for Landis parameter sexual maturity (traitID 155)
TRY_155 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 155) %>%
  dplyr::mutate(
    OrigValueStr = dplyr::na_if(.data$OrigValueStr, ""),  # Remove empty strings
    is_date = stringr::str_detect(.data$OrigValueStr, "\\d{4}-\\d{2}-\\d{2}"),  # Flag dates
    number_list = stringr::str_extract_all(.data$OrigValueStr, "\\d+\\.?\\d*"),  # Extract numbers
    OrigValueStr = sapply(.data$number_list, function(x) {  # Overwrite with cleaned value
      x_num <- suppressWarnings(as.numeric(x))
      if (length(x_num) == 0 || all(is.na(x_num))) return(NA_real_)
      else return(mean(x_num, na.rm = TRUE))
    })
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr), !.data$is_date) #keep only rows with valid values

#makes a column with mean age of flowering of trees, TRY misses Pseumenz and Larikaem
df_flower_age <- summarise_by_species(TRY_155, "mean", "Sexual Maturity") %>%
      dplyr::add_row(
          SpeciesCode = c("Pseumenz", "Larikaem"),
          "Sexual Maturity" = c(12, 10)
      )
Pseumenz_flowerage <- 12 #future possible pipeline, for now not possible
Larikaem_flowerage <- 10 #future possible pipeline, for now not possible

# Add missing species manually
df_flower_age_missing <- dplyr::tibble(
  SpeciesCode = c("Pseumenz", "Larikaem"),
  "Sexual Maturity" = c(Pseumenz_flowerage, Larikaem_flowerage)
)

# Combine both
df_flower_age <- dplyr::bind_rows(df_flower_age, df_flower_age_missing)

##prepare TRY database for Landis parameter plant longevity (traitID 59)
TRY_59 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 59) %>%
  dplyr::mutate(
    OrigValueStr = dplyr::na_if(.data$OrigValueStr, ""),  # Remove empty strings
    is_date = stringr::str_detect(.data$OrigValueStr, "\\d{4}-\\d{2}-\\d{2}"),  # Flag dates
    OrigValueStr = stringr::str_extract(.data$OrigValueStr, "\\d+\\.?\\d*"),  # Extract single number
    OrigValueStr = as.numeric(.data$OrigValueStr)  # Convert to numeric
  ) %>%
  dplyr::filter(!is.na("OrigValueStr"), !.data$is_date)  # Keep only rows with valid values

#makes a column with max age of a tree
df_plant_age <- summarise_by_species(TRY_59, "max", "Longevity")

##prepare TRY database for Landis parameter dispersal distance (traitID 193)
TRY_193 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 193) %>%
  dplyr::mutate(OrigValueStr = dplyr::na_if(.data$OrigValueStr, ""),  # Remove empty strings
                OrigValueStr = as.numeric(.data$OrigValueStr)  # Convert to numeric
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr))

#makes a column with effective dispersal distance, TRY misses Betuspec, Fagusylv, Piceabie, Querspec and Larikaem
df_eff_seed_dist <- TRY_193 %>% 
  dplyr::filter(!stringr::str_starts(.data$OriglName, "Max")) %>% # Exclude names starting with "Max"
  summarise_by_species(., "mean", "Seed Dispersal Dist Effective")
Betuspec_effseeddist <- 50 #future possible pipeline, for now not possible
Fagusylv_effseeddist <- 30 #future possible pipeline, for now not possible
Piceabie_effseeddist <- 45 #future possible pipeline, for now not possible
Querspec_effseeddist <- 100 #future possible pipeline, for now not possible
Larikaem_effseeddist <- 50 #future possible pipeline, for now not possible

# Add missing species manually
df_eff_seed_dist_missing <- dplyr::tibble(
  SpeciesCode = c("Betuspec", "Fagusylv", "Piceabie", "Querspec", "Larikaem"),
  "Seed Dispersal Dist Effective" = c(Betuspec_effseeddist, Fagusylv_effseeddist, Piceabie_effseeddist, Querspec_effseeddist, Larikaem_effseeddist)
) 

#complete effective_seed_distance
df_eff_seed_dist <- dplyr::bind_rows(df_eff_seed_dist, df_eff_seed_dist_missing)


#makes a column with maximum dispersal distance, TRY misses Querrubr, Betuspec, Fagusylv, Piceabie, Querspec and Larikaem
df_max_seed_dist <- TRY_193 %>% 
  dplyr::filter(stringr::str_starts(.data$OriglName, "Max")) %>% # Include names starting with "Max"
  dplyr::filter(!stringr::str_starts(.data$OriglName, "Eff")) %>% #Exclude names starting with "Effective"
  summarise_by_species(.,"max", "Seed Dispersal Dist Maximum")
Betuspec_maxseeddist <- 1000 #future possible pipeline, for now not possible
Fagusylv_maxseeddist <- 70 #future possible pipeline, for now not possible
Piceabie_maxseeddist <- 200 #future possible pipeline, for now not possible
Querspec_maxseeddist <- 1500 #future possible pipeline, for now not possible
Larikaem_maxseeddist <- 1000 #future possible pipeline, for now not possible
Querrubr_maxseeddist <- 1500 #future possible pipeline, for now not possible, this still needs a source!!!

# Add missing species manually
df_max_seed_dist_missing <- dplyr::tibble(
  SpeciesCode = c("Betuspec_maxseeddist", "Fagusylv_maxseeddist", "Piceabie_maxseeddist", "Querspec_maxseeddist", "Larikaem_maxseeddist", "Querrubr_maxseeddist"),
  "Seed Dispersal Dist Maximum" = c(Betuspec_maxseeddist, Fagusylv_maxseeddist, Piceabie_maxseeddist, Querspec_maxseeddist, Larikaem_maxseeddist, Querrubr_maxseeddist)
)

#complete effective_seed_distance
df_max_seed_dist <- dplyr::bind_rows(df_max_seed_dist, df_max_seed_dist_missing)  %>%
  dplyr::mutate(SpeciesCode = stringr::str_sub(.data$SpeciesCode, 1, 8))

##prepare TRY database for resprouting probability (Traitid 819)
TRY_819 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 819) %>%
  dplyr::filter(!stringr::str_detect(.data$OrigUnitStr, "%")) %>%
  dplyr::mutate(
    OrigValueStr = dplyr::na_if(.data$OrigValueStr, ""),  # Remove empty strings
    OrigValueStr = dplyr::case_when(
      .data$OrigValueStr %in% c("No", "no", "1") ~ "0.0", #0.0 for no chance for resprouting
      .data$OrigValueStr %in% c("Yes", "yes", "0", "moderate") ~ "0.8", #0.8 for all species that are able to resprout
      TRUE ~ .data$OrigValueStr
    ),
    OrigValueStr = as.numeric(.data$OrigValueStr)  # Convert to numeric
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr))  # Keep only rows with valid values

#makes a column with resprouting probability of trees
df_resprout_prob <- summarise_by_species(TRY_819, "min", "Vegetative Reprod Prob") #min because Larix kaempferi had an able to resprout data point, however that paper did not mention larix kaempferi
#also pinus sylvestris had an entry that had able to resprout, however, in that database the entry was not able to resprout
#also picea abies had an entry that had able to resprout, however, there was no source in the database, and other sources specified that picea abies was definitely not able to resprout

##makes a column with minimum resprouting age, TRY misses all species
Querrubr_minresprout <- 20 #future possible pipeline, for now not possible
Betuspec_minresprout <- 0 #future possible pipeline, for now not possible
Fagusylv_minresprout <- 20 #future possible pipeline, for now not possible
Pseumenz_minresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Piceabie_minresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Pinusylv_minresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Querspec_minresprout <- 20 #future possible pipeline, for now not possible
Larikaem_minresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0

#complete df minimum resprouting age
df_min_resprout_age <- dplyr::tibble(
  SpeciesCode = c("Querrubr", "Betuspec", "Fagusylv", "Pseumenz", "Piceabie", "Pinusylv", "Querspec", "Larikaem"),
  "Sprout Age Min" = c(Querrubr_minresprout, Betuspec_minresprout, Fagusylv_minresprout, Pseumenz_minresprout, Piceabie_minresprout, Pinusylv_minresprout, Querspec_minresprout, Larikaem_minresprout)
)

##makes a column with maximum resprouting age, TRY misses all species
Querrubr_maxresprout <- 200 #future possible pipeline, for now not possible
Betuspec_maxresprout <- 100 #future possible pipeline, for now not possible
Fagusylv_maxresprout <- 150 #future possible pipeline, for now not possible
Pseumenz_maxresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Piceabie_maxresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Pinusylv_maxresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0
Querspec_maxresprout <- 150 #future possible pipeline, for now not possible
Larikaem_maxresprout <- 0 #future possible pipeline, for now not possible, species that do not resprout need a 0

#complete df maximum resprouting age
df_max_resprout_age <- dplyr::tibble(
  SpeciesCode = c("Querrubr", "Betuspec", "Fagusylv", "Pseumenz", "Piceabie", "Pinusylv", "Querspec", "Larikaem"),
  "Sprout Age Max" = c(Querrubr_maxresprout, Betuspec_maxresprout, Fagusylv_maxresprout, Pseumenz_maxresprout, Piceabie_maxresprout, Pinusylv_maxresprout, Querspec_maxresprout, Larikaem_maxresprout)
)


##prepare TRY database for method of post fire regenaration (Traitid 318 and 819)
TRY_318.819 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID %in% c(318, 819)) %>%
  dplyr::mutate(
    OrigValueStr = dplyr::case_when( #resprout and serotiny are in different TRY traitID, serotiny takes priority over resprout over none
      .data$TraitID == 318 & .data$OriglName == "SeedlEmerg" & .data$OrigValueStr %in% c("low", "yes") ~ "serotiny",
      .data$TraitID == 819 & .data$OrigValueStr %in% c("No", "no", "1") ~ "none", #TRY paper uses 1 for no
      .data$TraitID == 819 & .data$OrigValueStr %in% c("Yes", "yes", "0", "moderate", "70") ~ "resprout", #TRY paper uses 0 for yes
      TRUE ~ NA_character_
    )
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr)) %>%
  dplyr::group_by(.data$SpeciesCode) %>%
  dplyr::summarize(
    OrigValueStr = dplyr::case_when(
      any(.data$OrigValueStr == "serotiny") ~ "serotiny",
      any(.data$OrigValueStr == "none") ~ "none",
      any(.data$OrigValueStr == "resprout") ~ "resprout"),
    .groups = "drop"
  )

df_postfire_regen <- TRY_318.819 #not with function, since value is not numerical


#make final input file for core species data
core_species_data <- df_plant_age %>%
  dplyr::full_join(df_flower_age, by = "SpeciesCode") %>%
  dplyr::full_join(df_eff_seed_dist, by = "SpeciesCode") %>%
  dplyr::full_join(df_max_seed_dist, by = "SpeciesCode") %>%
  dplyr::full_join(df_resprout_prob, by = "SpeciesCode") %>%
  dplyr::full_join(df_min_resprout_age, by = "SpeciesCode") %>%
  dplyr::full_join(df_max_resprout_age, by = "SpeciesCode") %>%
  dplyr::full_join(df_postfire_regen, by = "SpeciesCode")

core_species_data <- apply(core_species_data, 1, function(row) paste(row, collapse = "\t"))

header <- c(
  "LandisData Species"
)

#part that should make the output to the next NaaVRE block
output_lines <- c(header, core_species_data)
core_species_data <- "Core_species_data.txt"
writeLines(output_lines, core_species_data)

# ---- SppEcoregionData for biomass succession ----

#make an if loop, eventually this file should contain all the code for all species trait data that is required per 
#succession extension. So the following code will only run in case biomass succession is the succession extension that is chosen
 
#also read in establishment values from Probos, Probos misses Querrubr
df_prob_establish <- readr::read_csv(paste0(input_path, "establishmentTreeVeluwe.csv")) %>%
  dplyr::select(-1) %>%              # Remove first column if it's row names
  dplyr::summarise(across(everything(), sum, na.rm = TRUE)) %>%
  tidyr::pivot_longer(cols = everything(), names_to = "SpeciesCode", values_to = "ProbEstablish") %>%
  dplyr::mutate(
    Year = 0,
    EcoregionName = 101,
    ProbEstablish = pmin(.data$ProbEstablish / 8, 1)
  )

#add missing species manually
Querrubr_prob_establish <- 1

#make dataframe for ProbEstablishment missing
df_prob_establish_missing <- dplyr::tibble(
  Year = 0,
  EcoregionName = 101,
  SpeciesCode = c("Querrubr"),
  ProbEstablish = c(Querrubr_prob_establish)
)

#make final dataframe
df_prob_establish <- dplyr::bind_rows(df_prob_establish, df_prob_establish_missing)

#read in vitality scores from NBI 7
mortality <- readr::read_csv(paste0(input_path, "probMortality_biomass.txt"))

#make dataframe for Probmortality
df_prob_mortality <- mortality %>%
    dplyr::mutate(EcoregionName = 101,
                  Year = 0)

#prepare ANPPmax, comes from training algorythm, but for now default values
Pseumenz_anpp_max <- 7.12 #ANPPmax_pseumenz 
Larikaem_anpp_max <- 7.12 #ANPPmax_larikaem 
Piceabie_anpp_max <- 7.12 #ANPPmax_piceabie 
Pinusylv_anpp_max <- 7.12 #ANPPmax_pinusylv 
Fagusylv_anpp_max <- 11.3 #ANPPmax_fagusylv 
Querspec_anpp_max <- 11.3 #ANPPmax_querspec 
Querrubr_anpp_max <- 11.3 #ANPPmax_querrubr 
Betuspec_anpp_max <- 11.3 #ANPPmax_betuspec 

#make dataframe for ANPPmax
df_anpp_max <- dplyr::tibble(
  Year = 0,
  EcoregionName = 101,
  SpeciesCode = c("Larikaem", "Pseumenz", "Piceabie", "Pinusylv", "Fagusylv", "Querspec", "Querrubr", "Betuspec"),
  ANPPmax = c(Larikaem_anpp_max, Pseumenz_anpp_max, Piceabie_anpp_max, Pinusylv_anpp_max, 
              Fagusylv_anpp_max, Querspec_anpp_max, Querrubr_anpp_max, Betuspec_anpp_max)
)

#prepare BiomassMax, comes from training algorythm, but for now default values
Pseumenz_biomass_max <- 26000 #biomassMAX_pseumenz 
Larikaem_biomass_max <- 26000 #biomassMAX_larikaem
Piceabie_biomass_max <- 26000 #biomassMAX_piceabie
Pinusylv_biomass_max <- 26000 #biomassMAX_pinusylv
Fagusylv_biomass_max <- 26000 #biomassMAX_fagusylv
Querspec_biomass_max <- 26000 #biomassMAX_querspec
Querrubr_biomass_max <- 26000 #biomassMAX_querrubr
Betuspec_biomass_max <- 26000 #biomassMAX_betuspec

#make dataframe for BiomassMax
df_biomass_max <- dplyr::tibble(
  Year = 0,
  EcoregionName = 101,
  SpeciesCode = c("Larikaem", "Pseumenz", "Piceabie", "Pinusylv", "Fagusylv", "Querspec", "Querrubr", "Betuspec"),
  BiomassMax = c(Larikaem_biomass_max, Pseumenz_biomass_max, Piceabie_biomass_max, Pinusylv_biomass_max, 
              Fagusylv_biomass_max, Querspec_biomass_max, Querrubr_biomass_max, Betuspec_biomass_max)
) 

#make final input file for Species ecoregion Data for biomass
SppEcoregionData.biomass <- df_prob_establish %>%
  dplyr::full_join(df_prob_mortality, by = c("SpeciesCode", "Year", "EcoregionName")) %>%
  dplyr::full_join(df_anpp_max, by = c("SpeciesCode", "Year", "EcoregionName")) %>%
  dplyr::full_join(df_biomass_max, by = c("SpeciesCode", "Year", "EcoregionName"))

spp_ecoregion_data <- "SppEcoregionData.csv"
readr::write_csv(SppEcoregionData.biomass, spp_ecoregion_data)


# ---- SpeciesDataBiomass for biomass succession ----
#prepare TRY database for Landis parameter leaf longevity (traitID 12)
TRY_12 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 12) %>%
  dplyr::mutate(
    # Remove non-numeric characters from OrigValueStr
    OrigValueStr = stringr::str_extract(.data$OrigValueStr, "\\d+\\.?\\d*"),
    OrigValueStr = as.numeric(.data$OrigValueStr),
    
    # Normalize OrigUnitStr to lowercase for easier case_when matching
    OrigUnitStr = stringr::str_to_lower(.data$OrigUnitStr),
    
    # Apply unit conversion
    OrigValueStr = dplyr::case_when(
      .data$OrigUnitStr == "days" ~ .data$OrigValueStr / 365,
      .data$OrigUnitStr %in% c("month", "months") ~ .data$OrigValueStr / 12,
      .data$OrigUnitStr %in% c("year", "yr-1", "year-1") ~ .data$OrigValueStr,
      TRUE ~ NA_real_  # Handle unexpected units
    ),
    
    # Enforce minimum value of 1, because deciduous trees in landis should have a value of 1
    OrigValueStr = dplyr::if_else(.data$OrigValueStr < 1, 1, .data$OrigValueStr),
    
    # Standardize unit label
    OrigUnitStr = "year"
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr))  

#makes a column with mean longevity of leafs, TRY misses Larikaem
df_leaf_longevity <- summarise_by_species(TRY_12, "mean", "LeafLongevity")
Larikaem_leaf_longevity <- 1 #future possible pipeline, for now not possible

# Add missing species manually
df_leaf_longevity_missing <- dplyr::tibble(
  SpeciesCode = c("Larikaem"),
  LeafLongevity = c(Larikaem_leaf_longevity)
)

# Combine both
df_leaf_longevity <- dplyr::bind_rows(df_leaf_longevity, df_leaf_longevity_missing)  %>%
  dplyr::mutate(SpeciesCode = stringr::str_sub(.data$SpeciesCode, 1, 8))

#prepare TRY database for Landis parameter wood decay rate (traitID 1158)
TRY_1158 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 1158) %>%
  dplyr::filter(.data$OriglName == "single exponential model: Yt=Yoe-kt") %>%
  dplyr::filter(.data$OrigValueStr >= 0) %>%
  dplyr::mutate(OrigValueStr = as.numeric(.data$OrigValueStr))

#makes a column with mean of wood decay rate, k in Yt=Yoe-kt, TRY misses Larikaem
df_wood_decayrate <- summarise_by_species(TRY_1158, "mean", "WoodDecayRate")
Larikaem_wood_decayrate <- 0.021 #future possible pipeline, for now not possible

# Add missing species manually
df_wood_decayrate_missing <- dplyr::tibble(
  SpeciesCode = c("Larikaem"),
  WoodDecayRate = c(Larikaem_wood_decayrate)
)

# Combine both
df_wood_decayrate <- dplyr::bind_rows(df_wood_decayrate, df_wood_decayrate_missing)  %>%
  dplyr::mutate(SpeciesCode = stringr::str_sub(.data$SpeciesCode, 1, 8))



#prepare mortalitycurve
Pseumenz_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Larikaem_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Piceabie_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Pinusylv_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Fagusylv_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Querspec_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Querrubr_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age
Betuspec_mortality_curve <- 5 #future might be different because of different fit, for now 5, because of high maximum age

#make dataframe for mortalitycurve
df_mortality_curve <- dplyr::tibble(
  SpeciesCode = c("Larikaem", "Pseumenz", "Piceabie", "Pinusylv", "Fagusylv", "Querspec", "Querrubr", "Betuspec"),
  MortalityCurve = c(Larikaem_mortality_curve, Pseumenz_mortality_curve, Piceabie_mortality_curve, Pinusylv_mortality_curve, 
                     Fagusylv_mortality_curve, Querspec_mortality_curve, Querrubr_mortality_curve, Betuspec_mortality_curve)
)

#prepare growthcurve, should come from training algorythm, but for now default values
Pseumenz_growth_curve <- 0.9 #growthcurve_pseumenz
Larikaem_growth_curve <- 0.9 #growthcurve_larikaem
Piceabie_growth_curve <- 0.9 #growthcurve_piceabie
Pinusylv_growth_curve <- 0.9 #growthcurve_pinusylv
Fagusylv_growth_curve <- 0.9 #growthcurve_fagusylv
Querspec_growth_curve <- 0.9 #growthcurve_querspec
Querrubr_growth_curve <- 0.9 #growthcurve_querrubr
Betuspec_growth_curve <- 0.9 #growthcurve_betuspec

#make dataframe for growthcurve
df_growth_curve <- dplyr::tibble(
  SpeciesCode = c("Larikaem", "Pseumenz", "Piceabie", "Pinusylv", "Fagusylv", "Querspec", "Querrubr", "Betuspec"),
  GrowthCurve = c(Larikaem_growth_curve, Pseumenz_growth_curve, Piceabie_growth_curve, Pinusylv_growth_curve, 
                  Fagusylv_growth_curve, Querspec_growth_curve, Querrubr_growth_curve, Betuspec_growth_curve)
)

#prepare TRY database for Landis parameter leaf lignin content (traitID 87)
TRY_87 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 87) %>%
  dplyr::mutate(
    # Remove non-numeric characters from OrigValueStr
    OrigValueStr = as.numeric(.data$OrigValueStr)
    
  ) %>%
  dplyr::filter(!is.na(.data$OrigValueStr))  

#makes a column with mean of leaf lignin content, TRY misses Larikaem, Pseumenz, Piceabie, Fagusylv, Querspec, Querrubr
df_leaf_lignin <- summarise_by_species(TRY_87, "mean", "LeafLignin")
Larikaem_leaf_lignin <- 0.21 #future possible pipeline, for now not possible
Pseumenz_leaf_lignin <- 0.19 #future possible pipeline, for now not possible
Piceabie_leaf_lignin <- 0.18 #future possible pipeline, for now not possible
Fagusylv_leaf_lignin <- 0.15 #future possible pipeline, for now not possible
Querspec_leaf_lignin <- 0.167 #future possible pipeline, for now not possible
Querrubr_leaf_lignin <- 0.145 #future possible pipeline, for now not possible


# Add missing species manually
df_leaf_lignin_missing <- dplyr::tibble(
  SpeciesCode = c("Larikaem", "Pseumenz", "Piceabie", "Fagusylv", "Querspec", "Querrubr"),
  LeafLignin = c(Larikaem_leaf_lignin, Pseumenz_leaf_lignin, Piceabie_leaf_lignin, Fagusylv_leaf_lignin, Querspec_leaf_lignin, Querrubr_leaf_lignin))

# Combine both
df_leaf_lignin <- dplyr::bind_rows(df_leaf_lignin, df_leaf_lignin_missing)  %>%
  dplyr::mutate(SpeciesCode = stringr::str_sub(.data$SpeciesCode, 1, 8))

#prepare TRY database for Landis parameter shade tolerance (traitID 603)
TRY_603 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 603) %>%
  # Remove rows with "%" in OrigUnitStr
  dplyr::filter(!stringr::str_detect(.data$OrigUnitStr, "%")) %>%
  
  # Normalize OrigValueStr to lowercase for easier case_when usage
  dplyr::mutate(OrigValueStr = stringr::str_to_lower(.data$OrigValueStr)) %>%
  
  # Map textual values to numeric scale
  mutate(
    OrigValueStr = dplyr::case_when(
      .data$OrigValueStr %in% c("intolerant") ~ 1,
      .data$OrigValueStr %in% c("indifferent - intolerant") ~ 2,
      .data$OrigValueStr %in% c("intermediate") ~ 3,
      .data$OrigValueStr %in% c("indifferent") ~ 4,
      .data$OrigValueStr %in% c("tolerant") ~ 5,
      .data$OrigValueStr %in% as.character(1:9) ~ dplyr::case_when(
        .data$OrigValueStr == "1" ~ 5,
        .data$OrigValueStr == "2" ~ 4.5,
        .data$OrigValueStr == "3" ~ 4,
        .data$OrigValueStr == "4" ~ 3.5,
        .data$OrigValueStr == "5" ~ 3,
        .data$OrigValueStr == "6" ~ 2.5,
        .data$OrigValueStr == "7" ~ 2,
        .data$OrigValueStr == "8" ~ 1.5,
        .data$OrigValueStr == "9" ~ 1
      ),
      TRUE ~ NA_real_  # Remove other non-numeric or unknown values
    )
  ) %>%
  
  # Remove rows with NA values after transformation
  dplyr::filter(!is.na(.data$OrigValueStr)) %>%
  
  # Standardize OrigUnitStr label
  dplyr::mutate(OrigUnitStr = "tolerance_score")


#makes a column with mean of shade tolerance
df_shade_tolerance <- summarise_by_species(TRY_603, "mean", "ShadeTolerance") %>%
  dplyr::mutate(ShadeTolerance = round(.data$ShadeTolerance)) #Landis requires integer input in this category

#prepare TRY database for Landis parameter fire tolerance (traitID 318)
TRY_318 <- TRY_biomass %>%
  dplyr::filter(.data$TraitID == 318) %>%
  dplyr::filter(.data$OriglName != "SeedlEmerg") %>%
  # Step 2: Normalize OrigUnitStr to lowercase
  dplyr::mutate(OrigValueStr = stringr::str_to_lower(.data$OrigValueStr)) %>%
  
  # Map textual values to numeric scale
  dplyr::mutate(
    OrigValueStr = dplyr::case_when(
      .data$OrigValueStr %in% c("no", "none") ~ 1,
      .data$OrigValueStr %in% c("low", "surface fire") ~ 2,
      .data$OrigValueStr %in% c("yes") ~ 4,
      TRUE ~ NA_real_  # Remove other non-numeric or unknown values
    )
  ) %>%
  
  # Remove rows with NA values after transformation
  dplyr::filter(!is.na(.data$OrigValueStr)) %>%
  
  # Standardize OrigUnitStr label
  dplyr::mutate(OrigUnitStr = "tolerance_score")


#makes a column with mean of fire tolerance, TRY misses Querspec and Fausylv
df_fire_tolerance <- summarise_by_species(TRY_318, "mean", "FireTolerance") %>%
  dplyr::mutate(FireTolerance= round(.data$FireTolerance)) #Landis requires integer input in this category
Querspec_fire_tole <- 3 #future possible pipeline, for now not possible
Fagusylv_fire_tole <- 2 #future possible pipeline, for now not possible

# Add missing species manually
df_fire_tole_missing <- dplyr::tibble(
  SpeciesCode = c("Querspec", "Fagusylv"),
  FireTolerance = c(Querspec_fire_tole, Fagusylv_fire_tole)
)

# Combine both
df_fire_tolerance <- dplyr::bind_rows(df_fire_tolerance, df_fire_tole_missing)  %>%
  dplyr::mutate(SpeciesCode = stringr::str_sub(.data$SpeciesCode, 1, 8))


#make final input file for SpeciesData for biomass
SpeciesData.biomass <- df_leaf_longevity %>%
  dplyr::full_join(df_wood_decayrate, by = "SpeciesCode") %>%
  dplyr::full_join(df_mortality_curve, by = "SpeciesCode") %>%
  dplyr::full_join(df_growth_curve, by = "SpeciesCode") %>%
  dplyr::full_join(df_leaf_lignin, by = "SpeciesCode") %>%
  dplyr::full_join(df_shade_tolerance, by = "SpeciesCode") %>%
  dplyr::full_join(df_fire_tolerance, by = "SpeciesCode") 


species_data <- "SpeciesData.csv"
readr::write_csv(SpeciesData.biomass, species_data)

In [3]:
#Initial communities data retriever 
# ---
# NaaVRE:
#  cell:
#   outputs:
#    - nbi5_forestdata: String
#    - nbi5_sampletrees: String
#    - nbi5_plotdefinition: String
#   dependencies:
#    - name: Hmisc
# ...

# Load packages
install.packages("Hmisc")

library(Hmisc)

url <- "https://www.probos.nl/images/pdf/overig/NBI-2012-MFV-2006-v1.1.zip"

download.file(url, "/tmp/data/NBI.zip", mode = "wb")
unzip("/tmp/data/NBI.zip", exdir = "/tmp/data/NBI")

nbi5_sampletrees <- Hmisc::mdb.get("/tmp/data/NBI/NBI-2012-MFV-2006-v1.1.mdb", tables = "data_MFV_Proefbomen")
nbi5_forestdata <- Hmisc::mdb.get("/tmp/data/NBI/NBI-2012-MFV-2006-v1.1.mdb", tables = "data_MFV_boomgegevens")
nbi5_plotdefinition <- Hmisc::mdb.get("/tmp/data/NBI/NBI-2012-MFV-2006-v1.1.mdb", tables = "data_MFV_plotdefinitie")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



19 variables; Processing variable:1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 


In [4]:
#Initial communities data preprocessor 
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - nbi5_forestdata: String
#    - nbi5_sampletrees: String
#    - nbi5_plotdefinition: String
#   outputs:
#    - veluwe_sf: String 
#    - initial_communities_datafile: String
#    - Preprocessed_initial_communities_datafile: String
#   dependencies:
#    - name: dplyr
#    - name: tidyr
#    - name: stringr
#    - name: sf
#    - name: terra
# ...

#Install packages
install.packages("terra")
install.packages("sf")
#Load packages
library(dplyr)
library(tidyr)
library(stringr)
library(sf)
library(terra)

#Skim unnecessary columns 
nbi5_forestdata <- nbi5_forestdata[, c(1, 4, 7)]

#Create diameter in cm column 
nbi5_forestdata$diameter_cm <- nbi5_forestdata$DIAMETER / 10

#Rename
names(nbi5_forestdata)[names(nbi5_forestdata) == 'DIAMETER'] <- 'diameter_mm'
names(nbi5_forestdata)[names(nbi5_forestdata) == 'PLOTNUMMER'] <- 'plotnumber'
names(nbi5_forestdata)[names(nbi5_forestdata) == 'BOOMSOORT'] <- 'speciesname'

names(nbi5_sampletrees)[names(nbi5_sampletrees) == 'HOOGTE'] <- 'height_m'
names(nbi5_sampletrees)[names(nbi5_sampletrees) == 'PLOTNUMMER'] <- 'plotnumber'
names(nbi5_sampletrees)[names(nbi5_sampletrees) == 'BOOMSOORT'] <- 'speciesname'

standardise_nbi_keys <- function(df) {
  df %>%
    mutate(
      across(any_of(c("plotnumber", "speciesname")), as.character)
    )
}

nbi5_plotdefinition <- nbi5_plotdefinition %>%
  rename(plotnumber = PLOTNUMMER) %>%
  standardise_nbi_keys()

nbi5_forestdata <- nbi5_forestdata %>%
  standardise_nbi_keys()

nbi5_sampletrees <- nbi5_sampletrees %>%
  standardise_nbi_keys()

#Create table with growth factors
growthfactors <- data.frame(speciesname = c("Abeel", "Abies alba", "Abies grandis", "Amerikaanse eik", "Berk",
                                            "Beuk", "Corsicaanse den", "Corsikaanse den", "Douglas", "Es", 
                                            "Europese lariks", "Fijnspar", "Gewone esdoorn en spaanse aak", "Grove den",
                                            "Haagbeuk", "Iep", "Inlandse eik", "Italiaanse populier", "Japanse lariks",
                                            "Kastanje, Linde en Treurwilg", "Kastanje, Linde, Treurwilg", "Lijsterbes", 
                                            "Linde, kastanje en boswilg", "Omorikaspar", "Oostenrijkse den", "Pinus contorta", 
                                            "Populier", "Robinia", "Sitkaspar", "Tamme kastanje", "Thuja", "Trilpopulier", 
                                            "Tsuga","Weymouth den", "Wilg", "Zeeden", "Zoete kers", "Zwarte els"),
                            growthfactor = c("0.591", "0.984", "0.984", "1.180", "0.787", "0.984", "0.787", "0.787", "0.984",
                                             "0.984", "0.787", "0.984", "0.787", "0.787", "0.984", "0.591", "1.180", "0.591",
                                             "0.787", "0.867", "0.867", "1.050", "0.867", "0.984", "0.787", "0.787", "0.591",
                                             "0.750", "0.984", "1.180", "1.050", "0.591", "1.050", "0.787", "0.591", "0.787",
                                             "0.787", "0.591")) 

growthfactors <- growthfactors %>%
  dplyr::mutate(
    speciesname = as.character(speciesname),
    growthfactor = as.numeric(growthfactor)
  )

nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(speciesname = as.character(speciesname))

#Add the growth factor as new column in main file 
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::left_join(growthfactors, by = "speciesname")

#Add column with age by multiplying diameter and growth factor 
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(
    growthfactor = as.numeric(growthfactor),
    age = diameter_cm * growthfactor
  )

nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(speciesname = as.character(speciesname))

nbi5_sampletrees <- nbi5_sampletrees %>%
  dplyr::mutate(
    speciesname = as.character(speciesname))

#Merge the height from sampletrees into forestdata
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::left_join(
    nbi5_sampletrees %>% select(plotnumber, speciesname, height_m),
    by = c("plotnumber", "speciesname"),
      relationship = "many-to-many")

#There are N/As because not every species was measured in each plot 
#To deal with this, calculate average height per cm of the diameter for each species 
nbi5_sampletrees <- nbi5_sampletrees %>%
  dplyr::left_join(nbi5_forestdata %>% select(plotnumber, speciesname, diameter_cm),
            by = c("plotnumber", "speciesname"),
                  relationship = "many-to-many")

# Calculate species-specific height-per-cm ratio
height_per_cm <- nbi5_sampletrees %>%
  dplyr::filter(!is.na(height_m) & !is.na(diameter_cm)) %>%
  dplyr::mutate(height_per_cm = height_m / diameter_cm) %>%
  dplyr::group_by(speciesname) %>%
  dplyr::summarize(avg_height_per_cm = mean(height_per_cm, na.rm = TRUE)) %>%
  ungroup()

nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::left_join(height_per_cm, by = "speciesname")

#The trees that already had height from sampletrees keep that height, the other ones 
# get height by calculating height_m = diameter_cm * avg height per cm for that species
#delete the rows (about 15 rows) that have no height assigned to them 
nbi5_forestdata <- nbi5_forestdata %>%
  mutate(
    height_m = ifelse(is.na(height_m), diameter_cm * avg_height_per_cm, height_m)
  ) %>%
  filter(!is.na(height_m)) %>%
  select(plotnumber, speciesname, diameter_cm, growthfactor, age, height_m)

#Make cohorts
cohort_size <- 10  # years per cohort

# Assign cohort number to each tree
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(
    cohortage = ceiling(age / 10) * 10
  )

#collapse so that each plotnumber-speciesname-cohortage combination only appears once
nbi5_forestdata <- nbi5_forestdata %>%
  group_by(plotnumber, speciesname, cohortage) %>%
  summarise(
    n_trees    = n(),
    total_diam  = sum(diameter_cm, na.rm = TRUE),
    total_height = sum(height_m, na.rm = TRUE),
    mean_age   = mean(age, na.rm = TRUE),
    .groups = "drop"
  )

#remove the entries with these species names, too non-specific or cannot find data on growth factor etc. 
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::filter(
    !speciesname %in% c(
      "Naaldbomen, overige",
      "Chamaecyparis",
      "Overig inheems loof",
      "Inheems loof",
      "Overig uitheems loof",
      "Overige inheemse loofboomsoorten",
      "Overige uitheemse loofboomsoorten"
    )
  )

#Rename the species - if the species occurs less than 100 times, it is put into the rest category 
rename_map <- c(
  "Abeel" = "rest",
  "Abies alba" = "rest",
  "Abies grandis" = "rest", 
  "Amerikaanse eik" = "quercusrubra",
  "Berk" = "betula",
  "Beuk" = "fagussylvatica", 
  "Corsicaanse den" = "pinusnigralaricio",
  "Corsikaanse den" = "pinusnigralaricio", 
  "Den, overige" = "rest",
  "Douglas" = "pseudotsugamenziesii",
  "Es" = "fraxinusexcelsior",
  "Europese lariks" = "rest",
  "Fijnspar" = "piceaabies",
  "Gewone esdoorn en spaanse aak" = "acerpseudoplatanus",
  "Grove den" = "pinussylvestris",
  "Haagbeuk"= "rest",
  "Iep" = "rest",
  "Inlandse eik" = "quercusrobur",
  "Italiaanse populier" = "rest", 
  "Japanse lariks" = "larixkaempferi",
  "Kastanje, Linde en Treurwilg" = "rest",
  "Kastanje, Linde, Treurwilg" = "rest",
  "Lijsterbes" = "rest",
  "Linde, kastanje en boswilg" = "rest",
  "Omorikaspar" = "rest",
  "Oostenrijkse den" = "pinusnigranigra",
  "Pinus contorta" = "rest",
  "Populier" = "populus", 
  "Robinia" = "rest", 
  "Sitkaspar" = "rest",
  "Spar, overige" = "rest",
  "Tamme kastanje" = "rest",
  "Thuja" = "rest",
  "Trilpopulier" = "rest",
  "Tsuga" = "rest",
  "Weymouth den" = "rest",
  "Wilg" = "salix",
  "Zeeden" = "rest",
  "Zoete kers" = "rest",
  "Zwarte els" = "alnusglutinosa"
  
)

nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(
    speciesname = recode(speciesname, !!!rename_map)
  )

#biomass functions obtained from https://edepot.wur.nl/114235 
#rest = the BM function of salix/populus, thuja = den, tsuga = spar 
biomass_species <- list(
  "populusalba" = function(dbh, h) {
    0.240*dbh^2.22
  }, 
  "abiesalba" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951
  }, 
  "abiesgrandis" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951
  }, 
  "quercusrubra" = function(dbh, h) {
    0.056*dbh^1.963*h^0.794
  },  
  "betula" = function(dbh, h) {
    0.224*dbh^2.148
  },
  "fagussylvatica" = function(dbh, h) {
    0.224*dbh^2.148
  },   
  "pinusnigralaricio" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },  
  "pinus" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  }, 
  "fraxinusexcelsior" = function(dbh, h) {
    0.094*dbh^2.013*h^0.570
  }, 
  "pseudotsugamenziesii" = function(dbh, h) {
    0.111*dbh^2.396
  },
  "larixdecidua" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "piceaabies" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951 
  },
  "acerpseudoplatanus" = function(dbh, h) {
    0.051*dbh^2.256*h^0.532
  },
  "pinussylvestris" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "carpinusbetulus" = function(dbh, h) {
    0.051*dbh^2.256*h^0.532
  },
  "ulmus" = function(dbh, h) {
    0.240*dbh^2.220
  },
  "quercusrobur" = function(dbh, h) {
    0.056*dbh^1.963*h^0.794
  },
  "populusnigraitalica" = function(dbh, h) {
    0.240*dbh^2.220 
  },
  "larixkaempferi" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "rest" = function(dbh, h) {
    0.240*dbh^2.220 
  },
  "sorbus" = function(dbh, h) {
    0.207*dbh^2.074 
  },
  "piceaomorika" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951 
  },
  "pinusnigranigra" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "pinuscontorta" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "populus" = function(dbh, h) {
    0.240*dbh^2.220
  },
  "robinia" = function(dbh, h) {
    0.056*dbh^1.963*h^0.794
  },
  "piceasitchensis" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951
  },
  "picea" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951
  },
  "castaneasativa" = function(dbh, h) {
    0.056*dbh^1.963*h^0.794
  },
  "thuja" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "populustremula" = function(dbh, h) {
    0.240*dbh^2.220
  },
  "tsuga" = function(dbh, h) {
    0.123*dbh^1.473*h^0.951
  },
  "pinusstrobus" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "salix" = function(dbh, h) {
    0.240*dbh^2.22
  },
  "pinuspinaster" = function(dbh, h) {
    0.071*dbh^2.039*h^0.504
  },
  "prunusavium" = function(dbh, h) {
    0.094*dbh^2.013*h^0.570
  },
  "alnusglutinosa" = function(dbh, h) {
    0.224*dbh^2.148} 
  )

#calculate biomass in kg
nbi5_forestdata <- nbi5_forestdata %>%
  rowwise() %>%
  mutate(
    biomass_kg = biomass_species[[speciesname]](
      dbh = total_diam,     
      h = total_height
    )
  ) %>%
  ungroup()

#calculate the basal area to get to biomass in g/m2 eventually 
nbi5_forestdata <- nbi5_forestdata %>%
  dplyr::mutate(
    basalarea = pi*((10*total_diam)/2000)^2
  )

#calculate biomass in kg/m2. basal area from NBI is m2/ha 
nbi5_forestdata <- nbi5_forestdata %>%
  mutate(
    biomass_kg_per_m2 = biomass_kg/basalarea
  )

#LANDIS requires biomass in g/m2 
nbi5_forestdata$biomass_g_per_m2 <- nbi5_forestdata$biomass_kg_per_m2 * 1000  #in grams 
nbi5_forestdata$biomass_g_per_m2 <- as.integer(nbi5_forestdata$biomass_g_per_m2)
#------------------
#Prepare the correct shapefiles#

#The NBI grid originates from EEA, and contains 1km2 cells with a Northing and Easting. 
#Transform to EEA projection (3035) by calculating x and y coordinates 
nbi5_plotdefinition <- nbi5_plotdefinition %>%
  dplyr::mutate(
    GRID_N = as.integer(str_extract(GRID, "(?<=N)\\d+")),
    GRID_E = as.integer(str_extract(GRID, "(?<=E)\\d+")),
    x_3035 = GRID_E * 1000 + 500,
    y_3035 = GRID_N * 1000 + 500
  )

#Join plotdefinitie with preprocessed initial communities to add plot coordinates to forest data 
nbi5_plots_communities <- nbi5_plotdefinition %>%
  dplyr::left_join(
    nbi5_forestdata,
    by = "plotnumber"
  ) %>%
  filter(!is.na(speciesname))
#-----------------------------------------------------
#Load and transform the Natura2000 Veluwe shapefile
url_sf <- "https://service.pdok.nl/rvo/natura2000/atom/downloads/natura2000.gpkg"
temp_file <- tempfile(fileext = ".gpkg")
download.file(url_sf, destfile = temp_file, mode = "wb")

natura2000_sf <- sf::read_sf(temp_file) %>%
sf::st_transform(28992)

#alternative
#natura2000_sf <- sf::read_sf("https://service.pdok.nl/rvo/natura2000/atom/downloads/natura2000.gpkg") %>%
#sf::st_transform(28992)

natura2000_sf$objectid <- as.integer(natura2000_sf$objectid)

#create the veluwe sf by filtering the natura2000 shapefile 
veluwe_sf <- natura2000_sf %>%
  dplyr::filter(objectid == 4049)

#-------------------------------------------------
#Interpolate the plots to stretch over the Veluwe
#we have point plots with forest data: nbi5_plotdefinition
#now we need to make 100 by 100 m raster over veluwe 
# Convert Veluwe polygon to terra vector
veluwe_vect <- terra::vect(veluwe_sf)

# Create empty raster with 100x100 m cells
raster_for_veluwe <- terra::rast(
  veluwe_vect,
  resolution = 100,
  crs = "EPSG:28992"
)

#not interpolating all the forest data at once, but only plotnumber at first 
#first, make all the plots appear once in a new data set 
plots_unique <- nbi5_plots_communities %>%
  distinct(plotnumber, x_3035, y_3035)

#as shapefile with correct projection
plots_sf <- sf::st_as_sf(
  plots_unique,
  coords = c("x_3035", "y_3035"),
  crs = 3035
) |> 
  st_transform(28992)

#clip to veluwe 
plots_veluwe <- plots_sf[
  sf::st_within(plots_sf, veluwe_sf, sparse = FALSE),
]

#vectorize
plots_vect <- terra::vect(plots_veluwe)

#rasterize with nearest neighbor 
Veluwe_forest_raster <- rasterize(
  plots_vect,
  raster_for_veluwe,
  field = "plotnumber")

Veluwe_forest_raster <- mask(Veluwe_forest_raster, vect(veluwe_sf))

#so now we have the veluwe in a 100x100 raster filled with the plotnumber of the original plot that was 
#nearest the cell 
#the next step is to fill these cells with the other forest data, belonging to the plotnumber 
#to do this, we first transform Veluwe_forest_raster to a data frame
names(Veluwe_forest_raster) <- "plotnumber"

Veluwe_df <- as.data.frame(Veluwe_forest_raster, xy = TRUE)

#and then merge this with the forest information in preprocessed initial communities (renaming this 
#so that it forms the final document for NaaVre)
Preprocessed_initial_communities <- dplyr::left_join(
  Veluwe_df,
  nbi5_forestdata,
  by = "plotnumber"
)

#this file does not technically have communities yet: the Veluwe area has 518 plots  
#first create table with the assigned names based on the ranges for communities (divide 518 plots over 10 communities)
mapcodes <- data.frame(plotnumber = c("55427", "55745", "55746", "55747", "55748", "55749","55750","55751", "55752", "56056",
                                      "56071", "56073", "56074", "56076", "56078", "56079", "56082", "56380", "56381","56382",
                                      "56395", "56398", "56399", "56400", "56401", "56402", "56406", "56407", "56408", "56703", 
                                      "56708", "56718", "56719", "56720", "56722", "56723","56724", "56725", "56727", "56730", 
                                      "56734", "57022", "57023", "57025", "57027", "57028", "57032", "57043", "57044", "57045",
                                      "57046", "57047", "57049", "57050","57051", "57052", "57056", "57057", "57058", "57348",
                                      "57354", "57355", "57356", "57357", "57369", "57370", "57374", "57375", "57378", "57379",
                                      "57380", "57381","57382", "57383", "57384", "57669", "57671", "57673", "57680", "57681",
                                      "57682", "57691", "57693", "57695", "57696", "57707", "57708", "57709", "57995", "57996",
                                      "57997", "57998", "57999", "58006", "58015", "58016", "58017", "58018", "58023", "58024",
                                      "58029", "58031", "58032", "58033", "58324", "58325", "58327", "58328","58329", "58331",
                                      "58332", "58339", "58340", "58341", "58342", "58343", "58344", "58345", "58347", "58357",
                                      "58358", "58359", "58360", "58362", "58648", "58650", "58651", "58654", "58655", "58656",
                                      "58663", "58664", "58667", "58669", "58683", "58684", "58685", "58974", "58975", "58976",
                                      "58977", "58978", "58979", "58980", "58984", "58987", "58988", "58989", "58992", "58995", 
                                      "58996", "59005", "59010", "59011", "59300", "59302", "59304", "59306", "59308", "59310",
                                      "59313", "59320", "59327", "59328","59333", "59335", "59336", "59618", "59620", "59622", 
                                      "59623", "59625", "59629", "59630", "59636", "59639", "59641", "59644", "59645", "59647", 
                                      "59648", "59650", "59651", "59652", "59654", "59655", "59659", "59661", "59944", "59946", 
                                      "59953", "59955", "59957", "59959", "59960", "59961", "59968", "59970", "59971", "59972", 
                                      "59973", "59974", "59975", "59976","59978", "59979", "59986", "59987", "59989", "59990", 
                                      "60269", "60275", "60283", "60284","60286", "60287", "60291", "60293", "60296", "60297", 
                                      "60300", "60301", "60302", "60303","60304", "60305", "60306", "60307", "60308", "60309", 
                                      "60310", "60311", "60312", "60313","60596", "60597", "60598", "60600", "60601", "60604",
                                      "60607", "60608", "60609", "60613","60616", "60621", "60624", "60627", "60629", "60630", 
                                      "60631", "60633", "60634", "60635","60636", "60637", "60640", "60922", "60923", "60925",
                                      "60928", "60932", "60934", "60935","60936", "60937", "60938", "60939", "60941", "60945", 
                                      "60947", "60949", "60950", "60951","60952", "60953", "60954", "60955", "60957", "60958", 
                                      "60959", "60960", "60961", "60965","61261", "61262", "61263", "61264", "61265", "61266", 
                                      "61268", "61269", "61272", "61273","61274", "61275", "61276", "61277", "61278", "61280", 
                                      "61281", "61282", "61283", "61284","61285", "61291", "61292", "61573", "61574", "61576", 
                                      "61580", "61581", "61588", "61589","61590", "61591", "61592", "61593", "61594", "61595", 
                                      "61596", "61598", "61599", "61600","61601", "61602", "61603", "61604", "61605", "61608", 
                                      "61610", "61611", "61612", "61613","61616", "61618", "61898", "61900", "61905", "61912", 
                                      "61914", "61915", "61918", "61919","61920", "61921", "61922", "61923", "61924", "61925", 
                                      "61926", "61927", "61928", "61929","61930", "61931", "61933", "61935", "61936", "61941", 
                                      "61942", "62223", "62224", "62225","62231", "62235", "62236", "62237", "62239", "62240", 
                                      "62247", "62248", "62249", "62251","62252", "62254", "62255", "62256", "62258", "62259", 
                                      "62262", "62263", "62264", "62553","62556", "62557", "62559", "62560", "62561", "62562", 
                                      "62563", "62565", "62572", "62573","62586", "62587", "62588", "62589", "62590", "62593", 
                                      "62873", "62874", "62876", "62877","62915", "63198", "63199", "63203", "63204", "63206", 
                                      "63207", "63208", "63209", "63213","63237", "63238", "63239", "63241", "63246", "63523", 
                                      "63524", "63527", "63528", "63529","63530", "63531", "63532", "63534", "63535", "63536", 
                                      "63537", "63563", "63564", "63566","63849", "63853", "63854", "63855", "63856", "63857", 
                                      "63858", "63859", "63860", "63861","63862", "63888", "63890", "63891", "63892", "64172", 
                                      "64173", "64174", "64176", "64178","64220", "64499", "64501", "64504", "64505", "64506", 
                                      "64507", "64508", "64512", "64542","64543", "64544", "64545", "64546", "64547", "64548", 
                                      "64549", "64825", "64826", "64827", "64828", "64829", "64830", "64832", "64869", "64870", 
                                      "64873", "65148", "65149", "65151","65152", "65153", "65154", "65155", "65156", "65194",
                                      "65196", "65475", "65476", "65477","65478", "65479", "65480", "65801"),
                            mapcode = c("1","1","1","1","1","1","1","1","1","1","1",
                                        "2","2","2","2","2","2","2","2","2","2","2",
                                        "3","3","3","3","3","3","3","3","3","3","3",
                                        "4","4","4","4","4","4","4","4","4","4","4",
                                        "5","5","5","5","5","5","5","5","5","5","5",
                                        "6","6","6","6","6","6","6","6","6","6","6",
                                        "7","7","7","7","7","7","7","7","7","7","7",
                                        "8","8","8","8","8","8","8","8","8","8","8",
                                        "9","9","9","9","9","9","9","9","9","9","9",
                                        "10","10","10","10","10","10","10","10","10",
                                        "11","11","11","11","11","11","11","11","11",
                                        "12","12","12","12","12","12","12","12","12",
                                        "13","13","13","13","13","13","13","13","13",
                                        "14","14","14","14","14","14","14","14","14",
                                        "15","15","15","15","15","15","15","15","15",
                                        "16","16","16","16","16","16","16","16","16",
                                        "17","17","17","17","17","17","17","17","17",
                                        "18","18","18","18","18","18","18","18","18",
                                        "19","19","19","19","19","19","19","19","19",
                                        "20","20","20","20","20","20","20","20","20",
                                        "21","21","21","21","21","21","21","21","21","21",
                                        "22","22","22","22","22","22","22","22","22","22",
                                        "23","23","23","23","23","23","23","23","23","23",
                                        "24","24","24","24","24","24","24","24","24","24",
                                        "25","25","25","25","25","25","25","25","25","25",
                                        "26","26","26","26","26","26","26","26","26","26",
                                        "27","27","27","27","27","27","27","27","27","27",
                                        "28","28","28","28","28","28","28","28","28","28",
                                        "29","29","29","29","29","29","29","29","29","29",
                                        "30","30","30","30","30","30","30","30","30","30",
                                        "31","31","31","31","31","31","31","31","31","31",
                                        "32","32","32","32","32","32","32","32","32","32",
                                        "33","33","33","33","33","33","33","33","33","33",
                                        "34","34","34","34","34","34","34","34","34","34",
                                        "35","35","35","35","35","35","35","35","35","35",
                                        "36","36","36","36","36","36","36","36","36","36",
                                        "37","37","37","37","37","37","37","37","37","37",
                                        "38","38","38","38","38","38","38","38","38","38",
                                        "39","39","39","39","39","39","39","39","39","39",
                                        "40","40","40","40","40","40","40","40","40","40",
                                        "41","41","41","41","41","41","41","41","41","41",
                                        "42","42","42","42","42","42","42","42","42","42",
                                        "43","43","43","43","43","43","43","43","43","43",
                                        "44","44","44","44","44","44","44","44","44","44",
                                        "45","45","45","45","45","45","45","45","45","45",
                                        "46","46","46","46","46","46","46","46","46","46",
                                        "47","47","47","47","47","47","47","47","47","47",
                                        "48","48","48","48","48","48","48","48","48","48",
                                        "49","49","49","49","49","49","49","49","49","49",
                                        "50","50","50","50","50","50","50","50","50","50"
                            ))

mapcodes <- mapcodes %>%
  dplyr::mutate(plotnumber = as.numeric(plotnumber)) 

Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
dplyr::mutate(plotnumber = as.numeric(plotnumber))
              
#join the mapcodes with initial communities
Preprocessed_initial_communities <- Preprocessed_initial_communities %>%
  dplyr::left_join(mapcodes, by = "plotnumber")

#delete and shift columns to match LANDIS
initial_communities <- Preprocessed_initial_communities[,c(14,4,5,13)]
names(initial_communities)[names(initial_communities) == 'biomass_g_per_m2'] <- 'cohortbiomass'

Preprocessed_initial_communities_datafile <- "/tmp/data/Preprocessed_initial_communities.csv"
write.csv(Preprocessed_initial_communities, 
          file = Preprocessed_initial_communities_datafile, 
          row.names = FALSE)

initial_communities_datafile <- "/tmp/data/initial_communities.csv"
write.csv(initial_communities, 
          file = initial_communities_datafile, 
          row.names = FALSE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Hmisc’:

    src, summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.13.0, GDAL 3.10.2, PROJ 9.5.1; sf_use_s2() is TRUE

terra 1.8.93


Attaching package: ‘terra’


The following object is masked from ‘package:tidyr’:

    extract


The following objects are masked from ‘package:Hmisc’:

    describe, mask, zoom


Warning message:
“[set.cats] setting categories like this is deprecated; use a two-column data.frame instead”


In [7]:
#EcoregionsMap and InitialCommunitiesMap
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - veluwe_raster_ecoregions: String
#    - veluwe_sf: String
#    - Preprocessed_initial_communities_datafile: String
#   outputs:
#    - ecoregions_tif: String
#    - initial_communities_tif: String
#   dependencies:
#    - name: sf
#    - name: terra
# ...

veluwe_raster_ecoregions <- rasterize(veluwe_sf, 
                           raster_for_veluwe, 
                           field = "objectid", 
                           fun = "count")  

Ecoregions_tif <- "/tmp/data/veluwe_raster_ecoregions.tif"
writeRaster(veluwe_raster_ecoregions, 
            filename = "veluwe_raster_ecoregions.tif", 
            overwrite = TRUE)
#---
#Read the file 
Preprocessed_initial_communities <- readr::read_csv(Preprocessed_initial_communities_datafile, show_col_types = FALSE)

communities_sf <- st_as_sf(
  Preprocessed_initial_communities,
  coords = c("x", "y"),
  crs    = 3035)  

communities_sf <- st_transform(communities_sf, crs = 28992)

communities_sf$mapcode <- as.integer(communities_sf$mapcode)

communities_vect <- vect(communities_sf)

#create empty raster with 100x100 m cells
raster_for_communities <- rast(
  communities_vect,
  resolution = 100,                     
  crs        = "EPSG:28992")


#fill in with communities by mapcode
veluwe_raster_communities <- rasterize(
  x    = communities_vect,          
  y    = raster_for_communities,   
  field = "mapcode",               
  fun   = "first"            
)

Initial_communities_tif <- "/tmp/data/veluwe_raster_communities.tif"
writeRaster(veluwe_raster_communities, 
            filename = "veluwe_raster_communities.tif", 
            overwrite = TRUE)

In [15]:
#Configuration files
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - Preprocessed_KNMI_datafile: String
#    - Preprocessed_KNMI_spinup_datafile: String
#   outputs:
#    - ecoregions_txt: String
#    - climategenerator_txt: String
# ...

#produce ecoregions.txt
ecoregions_txt <- "/tmp/data/ecoregions.txt"
writeLines(c('LandisData  "Ecoregions"',
"",
">>         	  Map",
">> 	Active       Code  	      Name   	Description",
">> 	------       ---- 	     -----  	-----------",
'         yes          1	      eco1		 "managed"'),
           ecoregions_txt)
  
#produce Climategenerator.txt
climategenerator_txt <- "/tmp/data/climategenerator.txt"
writeLines(c('LandisData "Climate Config"',
            "ClimateTimeSeries ",                     param_climate_time_series,
            "ClimateFile ",                           Preprocessed_KNMI_datafile,
            "SpinUpClimateTimeSeries ",               param_spinup_climate_time_series,
            "SpinUpClimateFile ",                     Preprocessed_KNMI_spinup_datafile,
            "",
             "GenerateClimateOutputFiles		yes",
             "UsingFireClimate	            	no  << Optional parameter; default is no.",
             ">>FineFuelMoistureCode			100",
             ">>DuffMoistureCode			    100",
             ">>DroughtCode				        100",
             ">>FirstDayFire				    30",
             ">>LastDayFire				        320"), 
              climategenerator_txt)

ERROR: Error: object 'Preprocessed_KNMI_datafile' not found


In [16]:
#Biomass succession initialization file 
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - initial_communities_datafile: String
#    - initial_communities_tif: String
#    - climategenerator_txt: String
#    - species_datafile: String
#    - spp_ecoregion_datafile: String
#   outputs:
#    - biomass_succession_txt: String
# ...

#produce biomass-succession.txt
biomass_succession_txt <- "/tmp/data/biomass-succession.txt" 
writeLines(c('LandisData  "Biomass Succession"', 
"",
"",
">>------------------",
">> REQUIRED INPUTS",
">>------------------",
"",
"Timestep ", 			param_timestep,
"",
"SeedingAlgorithm  		WardSeedDispersal", 
"",
"InitialCommunities ",      initial_communities_datafile,
"InitialCommunitiesMap ",  	initial_communities_tif,
"ClimateConfigFile ",		climategenerator.txt,
"",
"CalibrateMode           no << optional parameter",
"SpinupMortalityFraction 0.001 << optional parameter",
"",
"",
">>----------------------------",
">> LIFE HISTORY PARAMETERS",
">>----------------------------",
"",
"MinRelativeBiomass",
">> Shade	Percent Max Biomass",
">> Class	by Ecoregions",
">> ----------	              --------------------",	
">>		eco1",
	"1	25%",
	"2	45%",
	"3	56%",
	"4  70%",
	"5	90%",
"",
"",
"SufficientLight",
">> Spp Shade	Probability",
">> Class	by Actual Shade",
">> ----------	--------------------",	
">> 		0 	     1 	     2 	     3       4 	     5",
   "1 		1.0 	0.5 	0.25 	0.0 	0.0 	0.0",
   "2 		1.0 	1.0 	0.5 	0.25 	0.0 	0.0",
   "3 		1.0 	1.0 	1.0 	0.5 	0.25 	0.0",
   "4 		1.0 	1.0 	1.0 	1.0 	0.5 	0.25",
   "5 		0.0 	0.0 	1.0 	1.0 	1.0 	1.0",
"",
"",
"SpeciesDataFile ",		species_data_datafile,
"",
"EcoregionParameters",
">>	AET (mm)",
"eco1	500",
"",
"SpeciesEcoregionDataFile ",  spp_ecoregion_datafile,
"",
"FireReductionParameters",
">>	Severity	WoodLitter	Litter",	
">>	Fire		Reduct		Reduct",	
	"1		      0.0		0.5",
	"2		      0.0		0.75",	
	"3		      0.0		1.0"), 
biomass_succession_txt)

ERROR: Error: object 'initial_communities_tif' not found


In [12]:
#Scenario file 
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - core_species_datafile: String
#    - ecoregions_txt: String
#    - ecoregions_tif: String
#    - biomass_succession_txt: String
#   outputs:
#    - scenario_txt: String
# ...

scenario_txt <- "/tmp/data/scenario.txt"
writeLines(c('LandisData  "Scenario"',
"", 
"",
"",
">>-------------------",
">> REQUIRED INPUTS",
">>-------------------",
"",
"Duration ",  	   param_duration,
"",
"Species ",  	   core_species_datafile,
"",
"Ecoregions ",      ecoregions_txt,
"EcoregionsMap ",   ecoregions_tif,
"",
"CellLength ", 	   param_cell_length,
"",
"",
"",
">> -----------------------",
">> SUCCESSION EXTENSIONS",
">> -----------------------",
"",
">> 	Succession Extension     Initialization File",
">> 	--------------------     -------------------",
'       "Biomass Succession"',    biomass_succession_txt,
"",
"",
">> --------------------------",
">> DISTURBANCE EXTENSIONS",
">> -------------------------",
"",
">> 	Disturbance Extension	Initialization File",
">>	    ---------------------	-------------------",
"",
"",
">>   DisturbancesRandomOrder  no  	<< optional << Commented (default) is 'no'",
"",                         	
"",
">> ------------------------",
">> OUTPUT EXTENSIONS",
">> ----------------------",
"",
">> 	Output Extension		Initialization File",
">> 	----------------		-------------------",
'  	    "Output Biomass"',		output_Biomass.txt,
"",
"",
"",
">> RandomNumberSeed  147        << optional parameter; uncomment for reproducibilty tests",
"                                << Commented (default) is a RandomNumberSeed generated using the current time"), 
"/tmp/data/scenario.txt")

ERROR: Error in parse(text = input): <text>:23:20: unexpected symbol
22: "",
23: "Duration "        param_duration
                       ^


In [43]:
# LANDIS II runner
install.packages("processx")
install.packages("terra")
library(processx)
library(terra)

# Get Biomass Succession code from GitHub
system("git clone https://github.com/LANDIS-II-Foundation/Extension-Biomass-Succession.git")

# Set working directory to Biomass Succession folder
setwd("Extension-Biomass-Succession/testings/CoreV8.0-BiomassSuccession7.0")

# Create function to track the progress of LANDIS-II in notebooks
system_with_progress <- function(cmd) {
    cb <- function(line, proc) {
        print(line)
        flush.console()
    }
    cmd <- strsplit(cmd, " ")[[1]]
    res <- processx::run(cmd[1], cmd[-1], stdout_line_callback=cb, stderr_line_callback=cb)
    return(res)
}

# Run LANDIS-II
res <- system_with_progress(paste("dotnet", Sys.getenv("LANDIS_CONSOLE"), "scenario_txt"))

# Visualise output (example)
x <- terra::rast("outputs/biomass/biomass-abiebals-0.tif")

[1] "/tmp/data/scenario.txt"
